# Solving Hydra(5): How Many Steps Would It Take to Slay a Five-Length Hydra?


## Nicholas S Georgescu

#### Published 2024-05-28 (originally [here](https://github.com/NGeorgescu/Hydra-Game/blob/main/Hydra.ipynb))

[This video](https://www.youtube.com/watch?v=prURA1i8Qj4) by Prof. Tom Crawford and Brady Haran featured on Numberphile has inspired this analysis.

In brief, the we want to know the number of cuts it takes to slay a hydra, which is done by recursively cutting heads at the lowest level greedily.  Each time a head is cut a number of heads appears at the grandfather node to the cut head equal to the number of cuts made up to this point in the slaying.  

A good data structure for this is a list to track the current number of heads and an integer for the current step count for the number of cuts.  Since heads are greedily removed down to one, we just need to know the number at each level, and we continue cutting and attaching one level up.

### setup for solving the problem

some imports that you don't need but might be handy

In [1]:
from tqdm import tqdm
from functools import lru_cache
import math
import sympy as sp

In case you need to increase the python printout size

In [2]:
# import sys
# sys.set_int_max_str_digits(4300)

we need to first define this function which will turn out to be useful

In [3]:
def f(x:int) -> int: 
    return (x+1)*2**(x-1)

[f(x) for x in [1,2,3,4,5,6,7,8,9,10]]

[2, 6, 16, 40, 96, 224, 512, 1152, 2560, 5632]

we then need to define another useful function which is defined as repeated applications of f

e.g. f(f(f(x)))=(x,3)

In [4]:
def a(x:int,n:int) -> int:
    for _ in range(n): x = f(x)
    return x

# in case you're wondering how quickly these get huge
[a(2,n) for n in [0,1,2,3]]

[2,
 6,
 224,
 3032994000054446976900039197289708450784178747535814404124156153036800]

let's define a function that calculates the number of chops of an actual hydra

In [5]:
def calculate_hydra(length, axial_count=1, initial_step=1, break_on_len_change=False):
    """
    calculates the actual hydra

    Parameters
    ----------
    length : int
        length of the initial hydra
    axial_count : int, option
        count of the topmost head, e.g. if [1,1,1,5] then axial_count=5 
    initial_step : int, optional
        initial step counter value (implying there's some history, if not then we start on the first step)
    break_on_len_change:bool, optional
        halts the operation if the length of the hydra changes

    Returns
    -------
    step : int
        returns the final step, or number of chops that have been performed.

    """
    hydra,step = [1]*(length-1)+[axial_count],initial_step # [1,1,1....,m],  initial step 
    while hydra:
        position, value = next(filter(lambda x: x[1]!=1, enumerate(hydra)), (None,None))
        # if position == len(hydra)-1 and step!=initial_step and break_on_axial: return hydra,step
        if value:
            hydra[position-1],hydra[position],step=(hydra[position-1]+step,hydra[position]-1,step+1) if position else (hydra[position-1],1,step+hydra[position]-1)
        else: # [1,1,1....,1]
            hydra.pop()
            if hydra: 
                hydra[-1],step=hydra[-1]+step,step+1
                if break_on_len_change: return hydra,step
            else: break
    return step

let's use this function to calculate the number of chops for hydras of increasing length

In [6]:
calculate_hydra(2)

3

In [7]:
calculate_hydra(3)

11

In [8]:
calculate_hydra(4)

1114111

so now the question: what is the value of `calculate_hydra(5)`?

to get an idea, let's break on len change with this number.  now `[1,1,1,1,1],1` is going to be equal to `[1,1,1,2],2`.  So let's do that:

In [9]:
calculate_hydra(4,2,2,break_on_len_change=True)

([1, 1, 22539988369408], 22539988369408)

so if we can find the `[1,1,x],y` for any arbitrary x and y for a three-length hydra then we're all set.  It should be noted that it's rather interesting that the number 22539988369408 isn't out of nowhere:

In [10]:
f(40)

22539988369408

### Solving a hydra of form [1,1,x],y
let's deal with x and y in a three-length hydra. let's deal with y first.  What we really want to know is how long it takes for [1,1,x],y to reach a hydra of [1,1,(x-1)].  Since this is independent of x, we can just solve `[1,1,1],y` for any y.

In [11]:
[calculate_hydra(2,i,i,break_on_len_change=True) for i in [1,2,3,4,5,6,7]]

[([2], 2),
 ([6], 6),
 ([16], 16),
 ([40], 40),
 ([96], 96),
 ([224], 224),
 ([512], 512)]

which is not so weird because if you'll recall:

In [12]:
[f(x) for x in [1,2,3,4,5,6,7]]

[2, 6, 16, 40, 96, 224, 512]

so for a hydra and initial step, of form `[1,1,1],y` is equivalent to `[1,f(y)],f(y)`.  We know that a hydra of form `[x],y` is equal to `x+y`.  A length-two hydra is left as an exercise to the reader, but will also depend on the f function above.

What this means that the final step is a function of `f(y)`.  What about increasing x?  Well if you'll recall, solving a hydra of form `[1,1,1],y` is equivalent to going from `[1,1,x],y` to `[1,1,(x-1)],s`.  So x just represents repeated applications of f.  Let's see if this pans out:

In [13]:
# calculate_hydra(length, axial_count, initial_step)

[calculate_hydra(3,1,y) for y in [1,2,3,4,5,6,7]]

[11, 31, 79, 191, 447, 1023, 2303]

In [14]:
[calculate_hydra(1,3,y) for y in [1,2,3,4,5,6,7]]

[3, 4, 5, 6, 7, 8, 9]

If you do some playing around with the f function you'll find that

In [15]:
all([
    calculate_hydra(3, axial_count, initial_step) 
    == 
    2*a(initial_step+1,axial_count)-1

    for axial_count, initial_step in
[(1,1),(2,1),(3,1),(1,2),(2,2),(1,3),(2,3),(1,4)]])

True

Since the hydra `[1,1,1,1,1], 1` is equal to the hydra  `[1,1,f(40)], f(40)` and a hydra of the form `[1,1,x], y` is equal to `2*a(y+1,x)-1`, then the five-length hydra is precisely the value `2*a(f(40)+1,f(40))-1`.

### How big is Hydra(5)?

How big is that? We can recognize that f(40) itself is, in fact, f(f(4)).  If we want to write this in up-arrow notation, we're looking at 2↑↑44, since:

In [16]:
int(math.log2(f(40)))

44

we also can approximate the f(x) function as 2↑x, since (x+1)>2 and 2**(x)>>(x+1) for values of x we care about.

Since f(f(4) = 2↑44, then applying 2↑ a number of 2↑44 times.  Thus an up-arrow-notation approximation for Hydra(5) is 2↑↑(2↑44).  If you want a more simplified arrow notation it's 2↑↑↑4 < Hydra(5) << 2↑↑↑5.

Even though a power tower of 2s that is 22 trillion tall seems incomprehensibly large, this makes it vastly smaller than even g1 of graham's number, which is 3↑↑↑↑3, which is absolutely gargantuan by comparison.